# Создание словаря для symspell

Создадим словарь на основе справочников профессий HH.RU и общероссийского классификатора занятий [ОК 010-2014 (МСКЗ-08)](https://data.mos.ru/classifier/7710168515-obshcherossiyskiy-klassifikator-zanyatiy?pageNumber=58&versionNumber=1&releaseNumber=1). Русскоязычная версия [ISCO08](https://esco.ec.europa.eu/en/classification/occupation_main).

Для каждого слова внесем в словарь все словоформы.

## Загрузка библиотек

In [51]:
import pandas as pd
import re

In [52]:
from symspellpy import SymSpell, Verbosity
import pymorphy2

In [53]:
import sys
sys.path.append('../')
from src import drop_stopwords, tokenize_drop_punkt, normalize_tokens
from src import unfold_abbreviation, read_abbreviations_dictionary, read_stopwords_dictionary

In [54]:
MA = pymorphy2.MorphAnalyzer()
ABBREVIATIONS = read_abbreviations_dictionary()
EXTRA_STOPWORDS = read_stopwords_dictionary()

## Загрузка данных

In [55]:
isco08 = pd.read_csv('../datasets/external/ok-010-2014_ISCO-08_ru.csv', encoding = 'cp1251', sep=';')
roles = pd.read_csv('../datasets/external/hh_prof_roles.csv')
specialities = pd.read_csv('../datasets/external/hh_prof_specializations.csv')

## Функции

In [56]:
def morh_analyse(word: str) -> list:
    result = []

    phrase = MA.parse(word)[0]
    tag = phrase.tag
    if 'LATN' in tag:
        result = [word]
    else:
        result = [p.word for p in phrase.lexeme]

    return result

## Формирование словаря SymSpell

In [57]:
docs = []
text = (isco08.NAME.to_list() 
        + roles.category_name.to_list() 
        + roles.prof_name.to_list()
        + specialities.category_name.to_list()
        + specialities.prof_name.to_list())

for sentence in text:
    sentence = ' '.join(unfold_abbreviation(sentence.split(), ABBREVIATIONS))
    tokens = drop_stopwords(tokenize_drop_punkt(sentence))
    for token in tokens:
        docs += morh_analyse(token)

dictionary = pd.Series(docs, name='term').value_counts().reset_index(level=0)
dictionary.columns = ['term', 'count']

Объединим словарь профессий со словарем symspell.

In [58]:
symdict = pd.read_csv('../models/symspell/ru-100k.txt', sep=' ', header=0)
symdict.columns = ['term', 'count']

In [59]:
merged_dict = symdict.merge(dictionary, how='outer', on='term', suffixes=('sym', 'prof'))


In [60]:
merged_dict.countsym.fillna(0, inplace=True)
merged_dict.countprof.fillna(0, inplace=True)
merged_dict['countprof'] *= 45000
merged_dict['count'] = (merged_dict['countsym'] + merged_dict['countprof']).astype('int')
merged_dict.drop(['countsym', 'countprof'], axis=1, inplace=True)

In [61]:
merged_dict.sort_values(by='count', ascending=False).to_csv('../models/symspell/professions.txt', sep=' ', header=False, index=False)

## Проверка загрузки

In [62]:
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = '../models/symspell/professions.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1)

True

In [63]:
input_term = 'инжинер'
suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, include_unknown=True)

In [64]:
for suggestion in suggestions:
    print(suggestion)

инженер, 1, 2348544


## Формирование справочника профессий

На основе справочника профессиональных ролей hh.ru сделаем спровочник профессий в формате

| Название профессии | Нормализованные токены | 
|-|-|
|Менеджер проектов|проект|
|...|...|

По нормализовнным токенам будем искать соответствие с ответом на вопрос "Кем ты работаешь?", а названия будем выводить в интерфейсе.

Для каждой строки в справочнике hh:
- выделим название роли, 
- разобъем названия по запятым, чтобы отдельно внести каждую профессию
- нормализуем токены

После этого объединим названия для одинаковых токенов. 

Результат сохраним.

In [65]:
labels = dict()
par_re = re.compile('(.+)\(([^\)]+)\)')
names =  []

# выделим каждое название в отдельный элемент списка
for role in roles.prof_name.unique(): 
    for n in [_.strip() for _ in role.split(',')]:
        # проверим сокращения в скобках
        match = par_re.search(n)
        if match:
            names.append(match.group(1).strip())
            names.append(match.group(2).strip())
        else:
            names.append(n)

In [66]:
# для каждого названия сделаем преобразование в нормальную форму
for n in names:
    text = ' '.join(unfold_abbreviation(n.split(), ABBREVIATIONS))
    tokens = tokenize_drop_punkt(text.lower())
    normalized_tokens = normalize_tokens(tokens)
    normalized_tokens = drop_stopwords(normalized_tokens, extra_stop_words=EXTRA_STOPWORDS) 
    if(len(normalized_tokens)):
        normalized_tokens = list(set(normalized_tokens))
        normalized_tokens.sort()
        tokenized = ' '.join(normalized_tokens)

        if tokenized not in labels:
            labels[tokenized] = []
        
        labels[tokenized].append(n.lower())




In [67]:
prof_keys = list(labels.keys())
prof_values = [ ', '.join(labels[pk]) for pk in prof_keys]
pd.DataFrame({'prof_tokens': prof_keys, 'prof_names': prof_values}).to_csv('../datasets/external/tokens-professions.csv', index=False)

In [68]:
print(len(prof_keys))

194


## Вывод

Мы создали и сохранили словарь для исправления опечаток в названиях профессий.